<div class="alert alert-block alert-success">
    <h1>
        Example notebook - AI governance control mappings
    </h1>
    <p>
        Link to dataset : Michael Brock Li' dataset
    </p>
</div>

## Executive Summary

This notebook demonstrates a **knowledge graph-based approach** to managing regulatory control mappings across multiple standards (ISO42001, ISO27001, ISO27701, EU AI Act, NIST RMF, SOC2).

**Key capabilities:**
- **Semantic search**: Find relevant controls using natural language queries
- **Cross-standard mapping**: Identify overlapping requirements between different frameworks
- **Graph visualization**: Explore relationships between domains, topics, controls, and standards
- **Hybrid search**: Combine semantic understanding with keyword matching for optimal results

---

## Data Overview

**Dataset**: Michael Brock Li's AI governance control mappings
- **44 control statements** covering AI governance requirements
- **6 regulatory standards** with reference mappings
- **Hierarchical structure**: Domain → Topic → Control → Standards

### Standard Coverage
- ISO 42001 (AI Management System)
- ISO 27001 (Information Security)
- ISO 27701 (Privacy Information Management)
- EU AI Act
- NIST RMF (Risk Management Framework)
- SOC 2 (Service Organization Control)


# Import modules and functions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings

warnings.filterwarnings("ignore")

In [3]:
import re
import pandas as pd
import networkx as nx
from sentence_transformers import SentenceTransformer
from IPython.display import display, Markdown

In [4]:
from turingdb.exceptions import TuringDBException

In [5]:
from turingdb_examples.utils import get_return_statements
from turingdb_examples.graph import build_create_command_from_networkx

In [6]:
from turingdb_kgsearch.embeddings import (
    build_node_only_embeddings,
    build_context_enriched_embeddings,
    build_smart_enriched_embeddings,
    build_sparse_embeddings,
    build_node2vec_embeddings,
)
from turingdb_kgsearch.search import (
    dense_search,
    sparse_search,
    print_results,
    hybrid_search,
    compare_search_methods,
)
from turingdb_kgsearch.subgraph import get_subgraph_around_query
from turingdb_kgsearch.visualization import (
    visualize_graph_with_pyvis,
    extract_and_visualize_subgraph,
)
from turingdb_kgsearch.workflow import (
    search_and_expand_hybrid_filtered,
    generate_report_hybrid_workflow_results,
)
from turingdb_kgsearch.statistics import get_subgraph_stats, print_subgraph_stats
from turingdb_kgsearch.ranking import (
    rank_nodes_by_importance,
    rank_nodes_by_importance_with_context,
    print_node_rankings,
    compare_node_importance,
    diagnose_rankings,
)
from turingdb_kgsearch.explain_results import (
    explain_retrieval,
    explain_top_results,
    print_explanation,
)
from turingdb_kgsearch.llm import (
    create_llm_prompt_with_graph,
    query_llm,
)

In [7]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

True

# Set path to data

In [8]:
example_name = "sec_8k_raw_text_filings_2024"
path_data = f"{os.getcwd()}/data/{example_name}"
if not os.path.exists(path_data):
    raise ValueError(f"{path_data} does not exists")

# Create graph using `turingdb` python package

<div class="alert alert-block alert-info">
    <h2>
        See <a href="https://docs.turingdb.ai/quickstart">TuringDB Get started documentation</a> for the important steps to follow :
    </h2>
    <h3>
        <ul>
            <li>Create your TuringDB account</li>
            <li>Create your instance in the <a href="https://console.turingdb.ai/auth">TuringDB Cloud UI</a></li>
            <li>Copy your Instance ID from the Database Instances management page</li>
            <li>Get API Key from the Settings in UI</li>
        </ul>
        Remember to have your instance active while working in this notebook !
    </h3>
</div>

## Connect to instance and transfer data

In [20]:
from turingdb import TuringDB

# Create TuringDB client
client = TuringDB(
    host="http://localhost:6666"  # Remove this parameter and set the two parameters below
    # instance_id=os.getenv("INSTANCE_ID"),
    # auth_token=os.getenv("AUTH_TOKEN"),
)

In [10]:
%%time

# Connect TuringDB client to s3 bucket
client.s3_connect(
    bucket_name="turing-internal",
    region="eu-west-2",
    access_key=os.getenv("AWS_ACCESS_KEY"),
    secret_key=os.getenv("AWS_SECRET_KEY"),
)

CPU times: user 250 ms, sys: 41.9 ms, total: 292 ms
Wall time: 2.36 s


## Import and format data

In [11]:
# Build Graph from SEC 8-K Filings Dataset
"""
Build Graph from SEC 8-K Filings Dataset

Assumes CSV has columns like:
- company_name / cik (company identifier)
- filing_date / accession_number
- item_numbers (8-K items like 1.01, 2.02, etc.)
- text / filing_text (the actual filing content)
- ticker / symbol
- industry / sector
"""

import pandas as pd
import networkx as nx
from datetime import datetime

def build_sec_8k_graph(df, sample_size=None):
    """
    Build knowledge graph from SEC 8-K filings.
    
    Graph structure:
        Company nodes → Filed → Filing nodes → Contains → Item nodes
        Company nodes → Industry → Industry nodes
        Filing nodes → MentionsCompany → Other Company nodes (if mentioned)
    
    Args:
        csv_path: Path to CSV file
        sample_size: Number of rows to process (None = all)
    
    Returns:
        NetworkX DiGraph
    """
    
    # Inspect columns
    print(f"Columns: {list(df.columns)}")
    if sample_size is not None:
        print(f"Keep only {sample_size} first rows of dataframe")
        df = df.iloc[:sample_size, :].copy()
        print(df.shape)
        
    
    # Create graph
    G = nx.DiGraph()
    
    # Build graph based on available columns
    for idx, row in df.iterrows():
        if idx % 100 == 0:
            print(f"Processing filing {idx}/{len(df)}...")
        
        # Extract key fields (adapt to actual column names)
        company_id = str(row.get('cik', row.get('company_name', f'company_{idx}')))
        company_name = str(row.get('company_name', row.get('ticker', company_id)))
        filing_id = f"filing_{row.get('accession_number', idx)}"
        filing_date = row.get('filing_date', row.get('date', ''))
        ticker = row.get('ticker', row.get('symbol', ''))
        industry = row.get('industry', row.get('sector', ''))
        items = row.get('item_numbers', row.get('items', ''))
        text = row.get('text', row.get('filing_text', ''))
        
        # 1. Company node
        company_node = f"company_{company_id}"
        if company_node not in G:
            G.add_node(
                company_node,
                type='company',
                name=company_name,
                ticker=ticker if pd.notna(ticker) else '',
                industry=industry if pd.notna(industry) else ''
            )
        
        # 2. Filing node
        G.add_node(
            filing_id,
            type='filing',
            date=str(filing_date) if pd.notna(filing_date) else '',
            text=str(text)[:500] if pd.notna(text) else '',  # Truncate for memory
            company=company_name,
            ticker=ticker if pd.notna(ticker) else ''
        )
        
        # 3. Company filed Filing
        G.add_edge(company_node, filing_id, rel='filed')
        
        # 4. Industry node (if present)
        if pd.notna(industry) and industry:
            industry_node = f"industry_{industry}"
            if industry_node not in G:
                G.add_node(industry_node, type='industry', name=industry)
            G.add_edge(company_node, industry_node, rel='belongs_to')
        
        # 5. Item nodes (8-K items like 1.01, 2.02, etc.)
        if pd.notna(items) and items:
            # Items might be comma-separated or single
            item_list = str(items).split(',') if ',' in str(items) else [str(items)]
            
            for item in item_list:
                item = item.strip()
                if item:
                    item_node = f"item_{item}"
                    item_description = get_8k_item_description(item)
                    
                    if item_node not in G:
                        G.add_node(
                            item_node,
                            type='item',
                            item_number=item,
                            description=item_description
                        )
                    
                    G.add_edge(filing_id, item_node, rel='contains')
    
    print(f"\n✓ Graph built: {G.number_of_nodes()} nodes, {G.number_of_edges()} edges")
    
    # Print breakdown
    node_types = {}
    for node in G.nodes():
        ntype = G.nodes[node].get('type', 'unknown')
        node_types[ntype] = node_types.get(ntype, 0) + 1
    
    print("\nNode breakdown:")
    for ntype, count in node_types.items():
        print(f"  {ntype}: {count}")
    
    return G


def get_8k_item_description(item_number):
    """Map 8-K item numbers to descriptions."""
    
    items = {
        '1.01': 'Entry into Material Agreement',
        '1.02': 'Termination of Material Agreement',
        '1.03': 'Bankruptcy or Receivership',
        '1.04': 'Mine Safety Disclosure',
        '1.05': 'Material Cybersecurity Incidents',
        '2.01': 'Completion of Acquisition or Disposition',
        '2.02': 'Results of Operations and Financial Condition',
        '2.03': 'Creation of Direct Financial Obligation',
        '2.04': 'Triggering Events',
        '2.05': 'Costs Associated with Exit Activities',
        '2.06': 'Material Impairments',
        '3.01': 'Notice of Delisting',
        '3.02': 'Unregistered Sales of Equity Securities',
        '3.03': 'Material Modifications to Rights',
        '4.01': 'Changes in Control',
        '4.02': 'Non-Reliance on Previously Issued Statements',
        '5.01': 'Changes in Control of Registrant',
        '5.02': 'Departure/Election of Directors or Officers',
        '5.03': 'Amendments to Articles/Bylaws',
        '5.04': 'Temporary Suspension of Trading',
        '5.05': 'Amendment to Code of Ethics',
        '5.06': 'Change in Shell Company Status',
        '5.07': 'Submission of Matters to Vote',
        '5.08': 'Shareholder Nominations',
        '7.01': 'Regulation FD Disclosure',
        '8.01': 'Other Events',
        '9.01': 'Financial Statements and Exhibits'
    }
    
    return items.get(item_number, f'Item {item_number}')


def enhance_graph_with_temporal(G):
    """Add temporal connections between filings."""
    
    # Group filings by company
    filings_by_company = {}
    for node in G.nodes():
        if G.nodes[node].get('type') == 'filing':
            company = None
            # Find company that filed this
            for pred in G.predecessors(node):
                if G.nodes[pred].get('type') == 'company':
                    company = pred
                    break
            
            if company:
                if company not in filings_by_company:
                    filings_by_company[company] = []
                
                filing_date = G.nodes[node].get('date', '')
                filings_by_company[company].append((node, filing_date))
    
    # Sort and connect sequential filings
    for company, filings in filings_by_company.items():
        # Sort by date
        filings.sort(key=lambda x: x[1])
        
        # Connect sequential filings
        for i in range(len(filings) - 1):
            G.add_edge(filings[i][0], filings[i+1][0], rel='preceded_by')
    
    print(f"✓ Added temporal connections")
    return G


def find_company_mentions(G, text_column='text'):
    """
    Find companies mentioned in filing text.
    Creates MentionsCompany edges.
    """
    
    # Get all company names/tickers
    companies = {}
    for node in G.nodes():
        if G.nodes[node].get('type') == 'company':
            name = G.nodes[node].get('name', '')
            ticker = G.nodes[node].get('ticker', '')
            if name:
                companies[name.lower()] = node
            if ticker:
                companies[ticker.upper()] = node
    
    # Search for mentions
    mentions_added = 0
    for node in G.nodes():
        if G.nodes[node].get('type') == 'filing':
            text = G.nodes[node].get(text_column, '').lower()
            
            if not text:
                continue
            
            # Find mentioned companies
            for company_identifier, company_node in companies.items():
                if company_identifier in text:
                    # Don't link to self
                    filing_company = None
                    for pred in G.predecessors(node):
                        if G.nodes[pred].get('type') == 'company':
                            filing_company = pred
                            break
                    
                    if filing_company != company_node:
                        G.add_edge(node, company_node, rel='mentions')
                        mentions_added += 1
    
    print(f"✓ Added {mentions_added} company mention edges")
    return G

In [12]:
%%time

path_turing_folder = f"{os.getenv('HOME')}/.turing"
gml_filename = f"{example_name}.gml"

if os.path.exists(f"{path_data}/{gml_filename}"):
    print(f"""✓ gml file "{gml_filename}" already generated and saved locally\n""")
    G = nx.read_gml(f"{path_data}/{gml_filename}")
    print(f"{G}\n")
    nx.write_gml(G, f"{path_data}/{example_name}.gml")
else:
    print("Create graph from csv")
    # Load and format csv
    csv_filename = f"{example_name}.csv"
    df = pd.read_csv(f"{path_turing_folder}/data/{csv_filename}")
    df = df.replace({r"\s+$": "", r"^\s+": ""}, regex=True).replace(r"\n", " ", regex=True).replace({r"\\": ""}, regex=True)
    print(f"✓ Loaded {len(df)} rows from CSV")
    display(df)

    # Build graph
    G = build_sec_8k_graph(
        df,
        #sample_size=3000  # Start with some rows for testing
    )
    
    # Enhance with temporal connections
    G = enhance_graph_with_temporal(G)
    print(G)
    
    # Save graph
    nx.write_gml(G, f"{path_data}/{example_name}.gml") # f"{path_turing_folder}/data/{example_name}.gml"

✓ gml file "sec_8k_raw_text_filings_2024.gml" already generated and saved locally

DiGraph with 63723 nodes and 110655 edges

CPU times: user 7.93 s, sys: 190 ms, total: 8.12 s
Wall time: 8.08 s


## Transfer gml to instance

In [13]:
%%time

gml_filename = f"{example_name}.gml"
client.transfer(
    src=f"data/{example_name}/{gml_filename}",
    dst=f"turingdb://{gml_filename}",  # to s3 bucket or TuringDB instance or local .turing
)

CPU times: user 262 ms, sys: 73.1 ms, total: 335 ms
Wall time: 1.49 s


## Import and load `turingdb` graph

In [21]:
graph_name = example_name
print(f"Graph name : '{graph_name}'")

Graph name : 'sec_8k_raw_text_filings_2024'


In [23]:
%%time

try:
    client.query(f"""
    IMPORT GRAPH {graph_name}
    FROM "{graph_name}.gml"
    """)
except TuringDBException as e:
    print(f"""{e}""")

CPU times: user 0 ns, sys: 2.9 ms, total: 2.9 ms
Wall time: 542 ms


In [24]:
try:
    client.load_graph(example_name)
except TuringDBException as e:
    print(f"""{e}: Graph "{example_name}" already loaded""")

GRAPH_LOAD_ERROR: Graph "sec_8k_raw_text_filings_2024" already loaded


In [25]:
%%time

# Set graph
client.set_graph(graph_name)

CPU times: user 7 μs, sys: 1e+03 ns, total: 8 μs
Wall time: 9.78 μs


<div class="alert alert-block alert-info">
    <h2>
        Visualize your graph in TuringDB Graph Visualizer ! Now that your instance is running:
    </h2>
    <h3>
        <ul>
            <li>Go to <a href="https://console.turingdb.ai/databases">TuringDB Console - Database Instances</a></li>
            <li>In your current instance panel, click on "Open Visualizer" button</li>
            <li>Visualizer opens, now you can choose your graph in the dropdown menu at the top-right corner</li>
        </ul>
        You can then play with your graph and visualize the nodes you want !
    </h3>
</div>

# Query TuringDB

## Use metaqueries to have insight on graph overall structure

<h3>
    To learn more about 📮 Metaqueries, please check TuringDB documentation on this <a href="https://turingdb.mintlify.app/query/cypher_subset#%F0%9F%93%AE-metaqueries">link</a>
</h3>

In [26]:
%%time

# CALL PROPERTIES() - returns a column of all the different node and edge properties and their types in the database
command = """
CALL PROPERTIES()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Property_ID", "Property_name", "Property_type"]
    display(df)

,Property_ID,Property_name,Property_type
0,0,label (String),String
1,1,type (String),String
2,2,name (String),String
3,3,ticker (String),String
4,4,industry (String),String
5,5,date (String),String
6,6,text (String),String
7,7,company (String),String
8,8,rel (String),String


CPU times: user 6.29 ms, sys: 29 μs, total: 6.32 ms
Wall time: 5.37 ms


In [27]:
%%time

# CALL LABELS () - returns a column of all the different node labels
command = """
CALL LABELS()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Node_type_ID", "Node_label"]
    display(df)

,Node_type_ID,Node_label
0,0,GMLNode


CPU times: user 3.78 ms, sys: 0 ns, total: 3.78 ms
Wall time: 3.15 ms


In [28]:
%%time

# CALL EDGETYPES() - returns a column of all the different edge types (edge equivalent of node labels)
command = """
CALL EDGETYPES()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Edge_type_ID", "Edge_label"]
    display(df)

,Edge_type_ID,Edge_label
0,0,GMLEdge


CPU times: user 3.93 ms, sys: 0 ns, total: 3.93 ms
Wall time: 3.31 ms


In [29]:
%%time

# CALL LABELSETS() - returns a two columns describing combinations of node labels
command = """
CALL LABELSETS()
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    df.columns = ["Node_type_ID", "Node_label"]
    display(df)

,Node_type_ID,Node_label
0,0,GMLNode


CPU times: user 3.73 ms, sys: 866 μs, total: 4.6 ms
Wall time: 4.01 ms


## Simple queries

In [30]:
%%time

# Match all edges and return them
command = """
MATCH (n)-[e]-(m)
RETURN n, e, m
"""
df = client.query(command)
if df.empty:
    print("No result found")
else:
    #df.columns = get_return_statements(command)
    display(df)

,0,1,2
0,0,0,1284
1,1,1,1181
2,2,2,5420
3,3,3,118
4,4,4,1269
...,...,...,...
110650,63718,110650,63142
110651,63719,110651,63132
110652,63720,110652,53828
110653,63721,110653,50591


CPU times: user 28.9 ms, sys: 2.99 ms, total: 31.9 ms
Wall time: 34.6 ms


# Load the embedding model

In [31]:
%%time

# This will convert text to vectors
model = SentenceTransformer("paraphrase-MiniLM-L3-v2")
print(f"✓ Model loaded: {model.get_sentence_embedding_dimension()} dimensions")

✓ Model loaded: 384 dimensions
CPU times: user 195 ms, sys: 10.2 ms, total: 205 ms
Wall time: 1.51 s


# Build vector index on the graph

## Vector Search Implementation - Dense (semantic) search

#### How It Works

Each control is converted to a **384-dimensional vector** using a pre-trained language model (`paraphrase-MiniLM-L3-v2`).

**Search process:**
1. Convert user query to vector
2. Calculate cosine similarity with all control vectors
3. Rank by similarity score (0-1)
4. Return top-k most relevant controls

#### Why Vectors?

- **Semantic understanding**: "data protection" matches "privacy safeguards"
- **Handles synonyms**: "AI governance" finds "artificial intelligence oversight"
- **No keyword dependency**: Works even without exact term matches

In [32]:
from turingdb_kgsearch.save_embeddings import load_embeddings, save_embeddings

In [33]:
# Define embedding paths
embeddings_folder = "embeddings"

### Use different approaches

In [35]:
%%time

# Node-only embeddings
method = "node_only"
embeddings_file_path = f"{path_data}/{embeddings_folder}/embeddings_{method}.npz"

if os.path.exists(embeddings_file_path):
    print(f"Loading pre-computed embeddings (method {method})...")
    node_vectors_node_only, node_texts_node_only, metadata_node_only, _ = load_embeddings(embeddings_file_path)
else:
    print(f"Building (method {method}) embeddings (this may take a while)...")
    node_vectors_node_only, node_texts_node_only = build_node_only_embeddings(G, model)
    save_embeddings(
        node_vectors_node_only,
        node_texts_node_only,
        filepath=embeddings_file_path,
        embedding_type="dense"
    )

print("\n" + "=" * 80 + "\n")

Loading pre-computed embeddings (method node_only)...
✓ Embeddings loaded from: /home/dev/turingdb-graphrag/notebooks/data/sec_8k_raw_text_filings_2024/embeddings/embeddings_node_only.npz
  - Type: dense
  - Format: Dense
  - 63723 vectors
  - Vector dimension: 384
  - Has texts: Yes
  - Has vectorizer: No


CPU times: user 1.15 s, sys: 28 ms, total: 1.18 s
Wall time: 1.18 s


In [37]:
%%time

# Context-enriched embeddings
strategy = "heavy"  # "lightweight"
method = f"context_enriched_{strategy}"
embeddings_file_path = f"{path_data}/{embeddings_folder}/embeddings_{method}.npz"

if os.path.exists(embeddings_file_path):
    print(f"Loading pre-computed embeddings (method {method})...")
    node_vectors_context_heavy, node_texts_context_heavy, metadata, _ = load_embeddings(embeddings_file_path)
else:
    print(f"Building (method {method}) embeddings (this may take a while)...")
    node_vectors_context_heavy, node_texts_context_heavy = build_context_enriched_embeddings(
        G, model, strategy=strategy
    )
    save_embeddings(
        node_vectors_context_heavy,
        node_texts_context_heavy,
        filepath=embeddings_file_path,
        embedding_type="dense"
    )

print("\n" + "=" * 80 + "\n")

Loading pre-computed embeddings (method context_enriched_heavy)...
✓ Embeddings loaded from: /home/dev/turingdb-graphrag/notebooks/data/sec_8k_raw_text_filings_2024/embeddings/embeddings_context_enriched_heavy.npz
  - Type: dense
  - Format: Dense
  - 63723 vectors
  - Vector dimension: 384
  - Has texts: Yes
  - Has vectorizer: No


CPU times: user 1.15 s, sys: 28 ms, total: 1.18 s
Wall time: 1.18 s


## Vector Search Implementation - Sparse (keyword) search

In [39]:
%%time

# Choose from which dense method take the created node_texts
node_texts = node_texts_context_heavy

# Build sparse embeddings
method = "sparse"
embeddings_file_path = f"{path_data}/{embeddings_folder}/embeddings_{method}.npz"

if os.path.exists(embeddings_file_path):
    print(f"Loading pre-computed embeddings (method {method})...")
    node_vectors_sparse, node_texts, metadata, vectorizer_sparse = load_embeddings(embeddings_file_path)
else:
    print(f"Building (method {method}) embeddings (this may take a while)...")
    node_vectors_sparse, _, vectorizer_sparse = build_sparse_embeddings(
        G=G, node_texts=node_texts  # max_features=500, ngram_range=(1, 2)
    )
    save_embeddings(
        node_vectors_sparse,
        node_texts,
        filepath=embeddings_file_path,
        embedding_type="sparse",
        vectorizer=vectorizer_sparse
    )

print("\n" + "=" * 80 + "\n")

Loading pre-computed embeddings (method sparse)...
✓ Embeddings loaded from: /home/dev/turingdb-graphrag/notebooks/data/sec_8k_raw_text_filings_2024/embeddings/embeddings_sparse.npz
  - Type: sparse
  - Format: Sparse
  - 63723 vectors
  - Vector dimension: 500
  - Has texts: Yes
  - Has vectorizer: Yes


CPU times: user 1.96 s, sys: 195 ms, total: 2.16 s
Wall time: 1.94 s


## Node2Vec

In [41]:
%%time

# Build Node2Vec embeddings
method = "node2vec"
embeddings_file_path = f"{path_data}/{embeddings_folder}/embeddings_{method}.npz"

if os.path.exists(embeddings_file_path):
    print(f"Loading pre-computed embeddings (method {method})...")
    node_vectors_node2vec, _, metadata, _ = load_embeddings(embeddings_file_path)
else:
    print(f"Building (method {method}) embeddings (this may take a while)...")
    if node_vectors_node2vec is None:
        node_vectors_node2vec = build_node2vec_embeddings(G, dimensions=384)
    save_embeddings(
        node_vectors_node2vec,
        None,
        embeddings_file_path,
        "node2vec"
    )

print("\n" + "=" * 80 + "\n")

Loading pre-computed embeddings (method node2vec)...
✓ Embeddings loaded from: /home/dev/turingdb-graphrag/notebooks/data/sec_8k_raw_text_filings_2024/embeddings/embeddings_node2vec.npz
  - Type: node2vec
  - Format: Dense
  - 63723 vectors
  - Vector dimension: 384
  - Has texts: No
  - Has vectorizer: No


CPU times: user 1.04 s, sys: 23.9 ms, total: 1.06 s
Wall time: 1.06 s


# Search capabilities

## Vector Search Implementation - Dense (semantic) search

Find controls relevant to any natural language query:

```python
results = search("data privacy protection", k=5)
```

**Use cases:**
- Exploratory research: "What controls cover AI model governance?"
- Concept-based lookup: "security monitoring requirements"
- Gap analysis: "What's missing in our risk management?"

### Query

In [43]:
%%time

# Try different queries
queries = [
    "Companies investing in AI",
    "Events related to AI regulation",
    "AI providers",
    "companies with risks of failure due to cloud providers",
    "AI model governance",
    "security monitoring requirements",
    "companies with risks linked to goods transportations/delivery",
]

for query in queries:
    print(f"\n{'=' * 80}")
    print(f"QUERY: '{query}'")
    print("=" * 80)
    results = dense_search(
        query=query,
        node_vectors=node_vectors_context_heavy,
        node_texts=node_texts,
        G=G,
        model=model,
        k=3
    )
    print_results(results)


QUERY: 'Companies investing in AI'

Found 3 results:

1. Similarity: 0.6553
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

2. Similarity: 0.6039
   Node: company_Investcorp AI Acquisition Corp.
   Type: company
   Text: company_Investcorp AI Acquisition Corp....

3. Similarity: 0.5871
   Node: company_AiAdvertising, Inc.
   Type: company
   Text: company_AiAdvertising, Inc....

QUERY: 'Events related to AI regulation'

Found 3 results:

1. Similarity: 0.4123
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

2. Similarity: 0.4049
   Node: company_Healthcare AI Acquisition Corp.
   Type: company
   Text: company_Healthcare AI Acquisition Corp....

3. Similarity: 0.3904
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

QUERY: 'AI providers'

Found 3 results:

1. Similarity: 0.7169
   Node: company_Healthcare AI Acquisition Corp.
   Type: company
   Text: company_Healthcare AI Acquisi

## Vector Search Implementation - Sparse (keyword) search

In [45]:
%%time

# Try different queries
for query in queries:
    print(f"\n{'=' * 80}")
    print(f"QUERY: '{query}'")
    print("=" * 80)
    results = sparse_search(
        query=query,
        node_vectors=node_vectors_sparse,
        node_texts=node_texts,  # same node texts used previously in dense search
        vectorizer=vectorizer_sparse,
        G=G,
        k=3
    )
    print_results(results)


QUERY: 'Companies investing in AI'

Found 3 results:

1. Similarity: 0.7194
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

2. Similarity: 0.7194
   Node: company_Jet.AI Inc.
   Type: company
   Text: company_Jet.AI Inc....

3. Similarity: 0.7194
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

QUERY: 'Events related to AI regulation'

Found 3 results:

1. Similarity: 1.0000
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

2. Similarity: 1.0000
   Node: company_Jet.AI Inc.
   Type: company
   Text: company_Jet.AI Inc....

3. Similarity: 1.0000
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

QUERY: 'AI providers'

Found 3 results:

1. Similarity: 1.0000
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

2. Similarity: 1.0000
   Node: company_Jet.AI Inc.
   Type: company
   Text: company_Jet.AI Inc....

3. Similarity: 1.000

## Hybrid search: Best of Both Worlds

### The Problem

- **Dense (semantic) search**: Great for concepts, misses exact terms
- **Sparse (keyword) search**: Finds exact matches, misses semantics

### The Solution

**Hybrid search** combines both approaches:

```
final_score = α × semantic_score + (1-α) × keyword_score
```

### Alpha Parameter Guide

| Alpha | Behavior | Best For |
|-------|----------|----------|
| 1.0 | Pure semantic | Conceptual queries |
| 0.7 | Favor semantics | General use (recommended) |
| 0.5 | Balanced | Mixed queries |
| 0.3 | Favor keywords | Technical lookups |
| 0.0 | Pure keywords | Exact term matching |

### When to Use What

**Dense (α=1.0)**
- Query: "What covers security?"
- Finds: Controls about protection, safeguards, defense

**Sparse (α=0.0)**
- Query: "ISO27001 A.8.24"
- Finds: Exact standard reference

**Hybrid (α=0.7)**
- Query: "NIST risk management frameworks"
- Finds: Both NIST references AND risk-related controls

### Query

In [48]:
%%time

# Try different queries
for query in queries:
    print(f"\n{'=' * 80}")
    print(f"QUERY: '{query}'")
    print("=" * 80)
    results = hybrid_search(
        query=query,
        G=G,
        dense_node_vectors=node_vectors_context_heavy,
        sparse_node_vectors=node_vectors_sparse,
        sparse_vectorizer=vectorizer_sparse,
        node_texts=node_texts,  # same node texts used for both dense and sparse
        model=model,
        k=3,
        alpha=0.7,  # 70% semantic, 30% keywords
    )
    print_results(results)


QUERY: 'Companies investing in AI'

Found 3 results:

1. Similarity: 1.0000
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

2. Similarity: 0.4919
   Node: company_Odysight.ai Inc.
   Type: company
   Text: company_Odysight.ai Inc....

3. Similarity: 0.3828
   Node: company_Investcorp AI Acquisition Corp.
   Type: company
   Text: company_Investcorp AI Acquisition Corp....

QUERY: 'Events related to AI regulation'

Found 3 results:

1. Similarity: 1.0000
   Node: company_Global AI, Inc.
   Type: company
   Text: company_Global AI, Inc....

2. Similarity: 0.6496
   Node: company_C3.ai, Inc.
   Type: company
   Text: company_C3.ai, Inc....

3. Similarity: 0.6117
   Node: company_NEURALBASE AI LTD.
   Type: company
   Text: company_NEURALBASE AI LTD....

QUERY: 'AI providers'

Found 3 results:

1. Similarity: 0.7000
   Node: company_Healthcare AI Acquisition Corp.
   Type: company
   Text: company_Healthcare AI Acquisition Corp....

2. Similarity: 0.

## Compare Dense vs Sparse vs Hybrid

In [49]:
queries[1]

'Events related to AI regulation'

In [58]:
%%time

# Limit results to specific node type (all node types by default)
k = 5
alpha = 0.7
node_type = None

for query in queries:
    compare_search_methods(
        query=query,
        G=G,
        dense_node_vectors=node_vectors_context_heavy,
        sparse_node_vectors=node_vectors_sparse,
        sparse_vectorizer=vectorizer_sparse,
        node_texts=node_texts,
        model=model,
        k=k,
        alpha=alpha,
        node_type=node_type
    )


QUERY: 'Events related to AI regulation'

1. DENSE ONLY (Semantic):
--------------------------------------------------------------------------------
1. 0.412 | company_Global AI, Inc....
2. 0.405 | company_Healthcare AI Acquisition Corp....
3. 0.390 | company_C3.ai, Inc....

2. SPARSE ONLY (Keywords):
--------------------------------------------------------------------------------
1. 1.000 | company_C3.ai, Inc....
2. 1.000 | company_Jet.AI Inc....
3. 1.000 | company_Global AI, Inc....

3. HYBRID (alpha=0.7):
--------------------------------------------------------------------------------
1. 1.000 (raw: D:0.41/S:1.00) | (norm: D:1.00/S:1.00) | company_Global AI, Inc....
2. 0.853 (raw: D:0.39/S:1.00) | (norm: D:0.79/S:1.00) | company_C3.ai, Inc....
3. 0.837 (raw: D:0.39/S:1.00) | (norm: D:0.77/S:1.00) | company_NEURALBASE AI LTD....


# Get subgraph around query results and visualise

### Graph-Based Context Retrieval

Get full context around relevant controls:

```python
subgraph = get_subgraph_around_query("risk management", k=3, hops=1)
```

**Returns:**
- Relevant controls
- Related topics and domains
- All mapped standard references
- Network connections

In [59]:
query = queries[1]

In [62]:
print(f"Query: '{query}'")

method_to_test = "hybrid"
possible_method_to_test = ["dense", "sparse", "hybrid"]
if method_to_test not in possible_method_to_test:
    raise ValueError(f"method_to_test has to be one of {possible_method_to_test}")

if method_to_test == "dense":
    # Dense search
    subgraph, results = get_subgraph_around_query(
        query=query,
        G=G,
        search_func=dense_search,
        search_params={
            "node_vectors": node_vectors_context_heavy,
            "node_texts": node_texts,
            "model": model
        },
        k=3,
        hops=1,
    )

elif method_to_test == "sparse":
    # Sparse search
    subgraph, results = get_subgraph_around_query(
        query=query,
        G=G,
        search_func=sparse_search,
        search_params={
            "sparse_vectors": node_vectors_sparse,
            "sparse_vectorizer": vectorizer_sparse,
            "node_texts": node_texts
        },
        k=3,
        hops=1,
    )

elif method_to_test == "hybrid":
    # Hybrid search
    subgraph, results = get_subgraph_around_query(
        query=query,
        G=G,
        search_func=hybrid_search,
        search_params={
            "dense_node_vectors": node_vectors_context_heavy,
            "sparse_node_vectors": node_vectors_sparse,
            "sparse_vectorizer": vectorizer_sparse,
            "node_texts": node_texts,  # same node texts used for both dense and sparse
            "model": model,
            "alpha": 0.7
        },
        k=3,
        hops=1,
    )

Query: 'Events related to AI regulation'


In [63]:
print(f"Query: '{query}'")
print(
    f"Subgraph: {subgraph.number_of_nodes()} nodes, {subgraph.number_of_edges()} edges"
)

# Show what's in the subgraph
types_in_subgraph = {}
for node, data in subgraph.nodes(data=True):
    ntype = data.get("type", "unknown")
    types_in_subgraph[ntype] = types_in_subgraph.get(ntype, 0) + 1

print("\nSubgraph composition:")
for ntype, count in types_in_subgraph.items():
    print(f"  {ntype}: {count}")

Query: 'Events related to AI regulation'
Subgraph: 17 nodes, 25 edges

Subgraph composition:
  filing: 14
  company: 3


# Visualization Capabilities

## Interactive Visualization (PyVis)

- Hover to see full control text
- Click to explore connections
- Physics-based layout
- Filterable and zoomable

**Features:**
- Color legend for node types
- Relevance-based sizing
- Relationship labels on edges
- Responsive browser-based interface

In [65]:
print(f"Query: '{query}'")

# With hybrid search
extract_and_visualize_subgraph(
    query=query,
    G=G,
    search_func=hybrid_search,
    search_params={
        "dense_node_vectors": node_vectors_context_heavy,
        "sparse_node_vectors": node_vectors_sparse,
        "sparse_vectorizer": vectorizer_sparse,
        "node_texts": node_texts,  # same node texts used for both dense and sparse
        "model": model,
        "alpha": 0.7
    },
    k=4,
    hops=2,
    output_file=f"{example_name}.html"  # f"{path_data}/{example_name}.html",
)

Query: 'Events related to AI regulation'
Auto-generated color map for 2 node types:
  company: #ff6b6b
  filing: #4ecdc4
✓ Interactive graph saved to: sec_8k_raw_text_filings_2024.html
  Nodes: 28
  Edges: 44


---

# TODO : Update from here

# Workflow

In [56]:
print(f"Query: '{query}'")

semantic_results, expanded, subgraph = search_and_expand_hybrid_filtered(
    query=query,
    G=G,
    node_vectors=node_vectors,
    node_texts=node_texts,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    structural_vectors=structural_vectors,
    model=model,
    k_search=5,
    max_hops=10,
    min_structural_sim=0.3,  # Must be structurally similar
    min_semantic_sim=0.3,  # AND semantically relevant to query
    structural_weight=0.5,  # 50-50 balance
    alpha=0.7,  # Weight alpha to attribute to semantic (dense) search, (1 - alpha) for keyword (sparse) search
)

report = generate_report_hybrid_workflow_results(semantic_results, expanded)
print(report)

Query: 'Events related to AI regulation'
Stage 1: Hybrid search for 'Events related to AI regulation'...
--------------------------------------------------------------------------------

Found 0 semantically relevant seed nodes:

Stage 2: Hybrid filtering (structural + semantic)...
  - Max hops: 10
  - Min structural similarity: 0.3
  - Min semantic similarity: 0.3
  - Structural weight: 0.5
--------------------------------------------------------------------------------

Building subgraph...
✓ Subgraph created:
  Total nodes: 0
  - Seed nodes: 0
  - Found neighbors: -5
  - Intermediate nodes: 0
  Total edges: 0
HYBRID-FILTERED WORKFLOW RESULTS


In [57]:
# Access subgraph data
print(f"\nSubgraph ({subgraph}) node attributes:")
for node in list(subgraph.nodes()):
    print(f"\n* {node}:")
    for key, value in subgraph.nodes[node].items():
        if key not in ["statement"]:  # Skip long text
            print(f"  {key}: {value}")

# Export subgraph if needed
# nx.write_gml(subgraph, "filtered_subgraph.gml")

# Visualise subgraph
visualize_graph_with_pyvis(subgraph, output_file="hybrid_filtered_graph.html")


Subgraph (DiGraph with 0 nodes and 0 edges) node attributes:
✓ Interactive graph saved to: hybrid_filtered_graph.html
  Nodes: 0
  Edges: 0


# Results exploration

## Graph statistics

In [58]:
# Usage
stats = get_subgraph_stats(
    subgraph, include_node_breakdown=True, include_centrality=True, include_paths=True
)
print_subgraph_stats(stats, verbose=True)

NetworkXPointlessConcept: Connectivity is undefined for the null graph.

## Node importance ranking

In [ ]:
node_to_check = "Amazon"
focus_type = "company"

In [ ]:
# Usage examples
rankings = rank_nodes_by_importance(
    subgraph,
    methods="all",  # or ['pagerank', 'degree', 'relevance']
    top_k=10,
    aggregate="average",  # or 'max' or {'pagerank': 0.4, 'degree': 0.3, 'relevance': 0.3}
)

print_node_rankings(rankings, subgraph, show_details=True)

In [ ]:
# Compare specific node
# Check if node exists first
if node_to_check in subgraph:
    node_comparison = compare_node_importance(node_to_check, rankings)
    if node_comparison:
        print(f"\nHow {node_to_check} ranks:")
        for method, info in node_comparison.items():
            print(f"  {method}: #{info['rank']} (score: {info['score']:.3f})")
    else:
        print(f"\{node_to_check} not in top rankings")
else:
    print(f"\{node_to_check} not in subgraph")

In [ ]:
filtered_rankings, full_rankings = rank_nodes_by_importance_with_context(
    subgraph,
    focus_type=focus_type,
    methods="all",
    top_k=10,
    aggregate="average",  # or 'max' or {'pagerank': 0.4, 'degree': 0.3, 'relevance': 0.3}
)

print("\n=== CONTROL NODE RANKINGS (computed on full graph) ===")
print_node_rankings(filtered_rankings, subgraph)

In [ ]:
# Debug why rankings might be zero for some nodes in some metrics
diagnose_rankings(subgraph, focus_type)

## Explain node retrieval

In [ ]:
# For a specific node
explanation = explain_retrieval(
    node_id=node_to_check,
    query=query,
    subgraph=subgraph,
    node_vectors=node_vectors,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    structural_vectors=structural_vectors,
    model=model,
)
print_explanation(explanation, verbose=True)

In [ ]:
explanations = explain_top_results(
    query=query,
    results=semantic_results,
    subgraph=subgraph,
    node_vectors=node_vectors,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    structural_vectors=structural_vectors,
    model=model,
    top_k=3,
)

# Take exploration results and ask LLM to answer original client query only using subgraph

In [119]:
api_keys = {
    "Anthropic": os.getenv("ANTHROPIC_API_KEY"),
    "Mistral": os.getenv("MISTRAL_API_KEY"),
    "OpenAI": os.getenv("OPENAI_API_KEY"),
}

## Create prompt

In [125]:
print(f"Query: '{query}'")

# Complete prompt
prompt = create_llm_prompt_with_graph(
    query=query,
    subgraph=subgraph,
    report=report,
    format="natural",  # or 'markdown'
)

# Send to LLM
print(prompt)

# Save to file
with open("llm_prompt.txt", "w") as f:
    f.write(prompt)

Query: 'companies with risks of failure due to cloud providers'
# Graph-Based Query Response

## User Query
"companies with risks of failure due to cloud providers"

## Search Results
HYBRID-FILTERED WORKFLOW RESULTS

1. SEED NODE (Hybrid Search Match):
   Node: Azure Cloud API
   Semantic Score: 1.000
   Text: Azure Cloud API...

   HYBRID-FILTERED NEIGHBORS:
   (Must pass BOTH structural AND semantic thresholds)

2. SEED NODE (Hybrid Search Match):
   Node: Microsoft
   Semantic Score: 0.430
   Text: Microsoft...

   HYBRID-FILTERED NEIGHBORS:
   (Must pass BOTH structural AND semantic thresholds)

3. SEED NODE (Hybrid Search Match):
   Node: Twilio
   Semantic Score: 0.315
   Text: Twilio...

   HYBRID-FILTERED NEIGHBORS:
   (Must pass BOTH structural AND semantic thresholds)

4. SEED NODE (Hybrid Search Match):
   Node: Ride Sharing Services
   Semantic Score: 0.290
   Text: Ride Sharing Services...

   HYBRID-FILTERED NEIGHBORS:
   (Must pass BOTH structural AND semantic threshold

## Query LLM

In [126]:
%%time

provider = "Anthropic"

result = query_llm(
    prompt=prompt,
    # system_prompt=system_prompt,
    provider=provider,
    api_key=api_keys[provider],
    temperature=0.2,
)

CPU times: user 23.2 ms, sys: 2.01 ms, total: 25.2 ms
Wall time: 7.94 s


In [127]:
display(Markdown(result))

Based on the provided graph data, here's an analysis of companies with risks of failure due to cloud providers:

Key Insights:
1. Cloud Provider Dependency:
- Microsoft and Twilio are both connected to Azure Cloud API
- This suggests potential vulnerability to cloud service disruptions
- Both companies rely on the same cloud infrastructure (Azure)

2. Risk Patterns:
- Companies like Microsoft and Twilio demonstrate direct dependency on cloud APIs
- The graph reveals a potential single point of failure through Azure Cloud API
- Ride Sharing Services appears as a customer segment, which might also be impacted by cloud service risks

3. Structural Observations:
- Limited graph connectivity (only 2 edges)
- Semantic scores vary, with Azure Cloud API having the highest relevance (1.000)
- Amazon is present in the graph but not directly linked to the cloud API

Potential Risks:
- Concentration risk: Multiple companies depending on a single cloud provider (Azure)
- Service disruption could simultaneously impact multiple organizations
- Limited graph data suggests more comprehensive analysis would be beneficial

Limitations of Analysis:
- Small sample size (5 nodes)
- Incomplete relationship mapping
- Lack of detailed risk assessment information

Recommendation:
Companies should consider:
- Multi-cloud strategies
- Redundancy in cloud service providers
- Robust disaster recovery plans

Note: This analysis is strictly based on the provided graph data and may not represent a comprehensive risk assessment.

In [128]:
print("Notebook finished !")

Notebook finished !


---

# Load the embedding model

In [24]:
%%time

# This will convert text to vectors
model = SentenceTransformer("paraphrase-MiniLM-L3-v2")
print(f"✓ Model loaded: {model.get_sentence_embedding_dimension()} dimensions")

✓ Model loaded: 384 dimensions
CPU times: user 176 ms, sys: 16.9 ms, total: 193 ms
Wall time: 1.43 s


# Build vector index on the graph

## Vector Search Implementation - Dense (semantic) search

#### How It Works

Each control is converted to a **384-dimensional vector** using a pre-trained language model (`paraphrase-MiniLM-L3-v2`).

**Search process:**
1. Convert user query to vector
2. Calculate cosine similarity with all control vectors
3. Rank by similarity score (0-1)
4. Return top-k most relevant controls

#### Why Vectors?

- **Semantic understanding**: "data protection" matches "privacy safeguards"
- **Handles synonyms**: "AI governance" finds "artificial intelligence oversight"
- **No keyword dependency**: Works even without exact term matches

### Use the three different approaches

In [30]:
%%time

# Build different versions
node_only = build_node_only_embeddings(G, model)  # Node-only embeddings
lightweight = build_context_enriched_embeddings(
    G, model, strategy="lightweight"
)  # Context-enriched embeddings

print("\n" + "=" * 80 + "\n")

ERROR! Session/line number was not unique in database. History logging moved to new session 192
✓ Vector index built using node-only embeddings approach: 63723 vectors
✓ Vector index built using context-enriched embeddings approach (strategy lightweight): 63723 vectors


CPU times: user 8min 58s, sys: 17.3 s, total: 9min 15s
Wall time: 33 s


## Vector Search Implementation - Sparse (keyword) search

In [31]:
%%time

# Build sparse embeddings
sparse_vectors, node_texts, sparse_vectorizer = build_sparse_embeddings(
    G=G, max_features=500, ngram_range=(1, 2)
)

Building sparse index (TF-IDF)...
✓ Sparse index built: 63723 vectors
  Vocabulary size: 500
  Sample terms: ['acquisition', 'acquisition corp', 'acquisition ii', 'acquisitions', 'acquisitions corp', 'advanced', 'aerospace', 'agriculture', 'ai', 'ai corp', 'ai holdings', 'air', 'airlines', 'alliance', 'america', 'american', 'amt', 'amt free', 'asset', 'asset high']
CPU times: user 2.35 s, sys: 403 ms, total: 2.75 s
Wall time: 2.26 s


## Node2Vec

# Search capabilities

## Vector Search Implementation - Dense (semantic) search

Find controls relevant to any natural language query:

```python
results = search("data privacy protection", k=5)
```

**Use cases:**
- Exploratory research: "What controls cover AI model governance?"
- Concept-based lookup: "security monitoring requirements"
- Gap analysis: "What's missing in our risk management?"

In [31]:
# Choose most relevant approach
node_vectors = smart[0]
node_texts = smart[1]

### Query

In [32]:
%%time

# Example 1: Find controls about data privacy
query = "data privacy protection"
results = dense_search(
    query=query,
    node_vectors=node_vectors,
    node_texts=node_texts,
    G=G,
    model=model,
    k=3,
    # node_type='control'
)
print(f"Query: '{query}'")
print_results(results)

Query: 'data privacy protection'

Found 3 results:

1. Similarity: 0.7615
   Node: domain_Privacy
   Type: domain
   Text: Domain: Privacy. Topics: Privacy by Design and Governance, Personal Data Management, Privacy Compliance and Monitoring, Privacy-Enhancing Technologies and Mechanisms...

2. Similarity: 0.6742
   Node: topic_Privacy-Enhancing Technologies and Mechanisms
   Type: topic
   Text: Topic: Privacy-Enhancing Technologies and Mechanisms. Domain: Privacy. Examples: The organisation shall implement appropriate technical mechanisms and privacy-en...

3. Similarity: 0.6605
   Node: control_30
   Type: control
   Text: The organisation shall implement appropriate technical mechanisms and privacy-enhancing technologies in AI systems to protect personal data and ensure privacy by default. This includes implementing en...
   Domain: Privacy
   Topic: Privacy-Enhancing Technologies and Mechanisms
CPU times: user 79.6 ms, sys: 1.55 ms, total: 81.2 ms
Wall time: 6.59 ms


In [33]:
%%time

# Other examples
# Try different queries
queries = [
    "AI model governance",
    "risk management controls",
    "security monitoring requirements",
    "data subject rights",
]

for query in queries:
    print(f"\n{'=' * 80}")
    print(f"QUERY: '{query}'")
    print("=" * 80)
    results = dense_search(
        query="data privacy protection",
        node_vectors=node_vectors,
        node_texts=node_texts,
        G=G,
        model=model,
        k=3,
        # node_type='control'
    )
    print_results(results)


QUERY: 'AI model governance'

Found 3 results:

1. Similarity: 0.7615
   Node: domain_Privacy
   Type: domain
   Text: Domain: Privacy. Topics: Privacy by Design and Governance, Personal Data Management, Privacy Compliance and Monitoring, Privacy-Enhancing Technologies and Mechanisms...

2. Similarity: 0.6742
   Node: topic_Privacy-Enhancing Technologies and Mechanisms
   Type: topic
   Text: Topic: Privacy-Enhancing Technologies and Mechanisms. Domain: Privacy. Examples: The organisation shall implement appropriate technical mechanisms and privacy-en...

3. Similarity: 0.6605
   Node: control_30
   Type: control
   Text: The organisation shall implement appropriate technical mechanisms and privacy-enhancing technologies in AI systems to protect personal data and ensure privacy by default. This includes implementing en...
   Domain: Privacy
   Topic: Privacy-Enhancing Technologies and Mechanisms

QUERY: 'risk management controls'

Found 3 results:

1. Similarity: 0.7615
   Node: domai

## Vector Search Implementation - Sparse (keyword) search

In [34]:
%%time

# Example: Find controls about data privacy
query = "data privacy protection"
results = sparse_search(
    query=query,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    node_texts=node_texts,
    G=G,
    k=3,
    node_type="control",
)
print(f"Query: '{query}'")
print_results(results)

# Try different queries
queries = [
    "AI model governance",
    "risk management controls",
    "security monitoring requirements",
    "data subject rights",
]

for query in queries:
    print(f"\n{'=' * 80}")
    print(f"QUERY: '{query}'")
    print("=" * 80)
    results = sparse_search(
        query=query,
        sparse_vectors=sparse_vectors,
        sparse_vectorizer=sparse_vectorizer,
        node_texts=node_texts,
        G=G,
        k=3,
        node_type="control",
    )
    print_results(results)

Query: 'data privacy protection'

Found 3 results:

1. Similarity: 0.4642
   Node: control_27
   Type: control
   Text: The organisation shall implement privacy by design principles in all AI systems, ensuring privacy considerations are embedded from initial planning through system retirement. This includes establishin...
   Domain: Privacy
   Topic: Privacy by Design and Governance

2. Similarity: 0.4379
   Node: control_30
   Type: control
   Text: The organisation shall implement appropriate technical mechanisms and privacy-enhancing technologies in AI systems to protect personal data and ensure privacy by default. This includes implementing en...
   Domain: Privacy
   Topic: Privacy-Enhancing Technologies and Mechanisms

3. Similarity: 0.3954
   Node: control_29
   Type: control
   Text: The organisation shall establish processes to monitor compliance with privacy requirements, detect and respond to privacy incidents, and ensure continuous improvement of privacy controls. This incl

## Hybrid search: Best of Both Worlds

### The Problem

- **Dense (semantic) search**: Great for concepts, misses exact terms
- **Sparse (keyword) search**: Finds exact matches, misses semantics

### The Solution

**Hybrid search** combines both approaches:

```
final_score = α × semantic_score + (1-α) × keyword_score
```

### Alpha Parameter Guide

| Alpha | Behavior | Best For |
|-------|----------|----------|
| 1.0 | Pure semantic | Conceptual queries |
| 0.7 | Favor semantics | General use (recommended) |
| 0.5 | Balanced | Mixed queries |
| 0.3 | Favor keywords | Technical lookups |
| 0.0 | Pure keywords | Exact term matching |

### When to Use What

**Dense (α=1.0)**
- Query: "What covers security?"
- Finds: Controls about protection, safeguards, defense

**Sparse (α=0.0)**
- Query: "ISO27001 A.8.24"
- Finds: Exact standard reference

**Hybrid (α=0.7)**
- Query: "NIST risk management frameworks"
- Finds: Both NIST references AND risk-related controls

### Query

In [35]:
%%time

# Example: Find controls about data privacy
query = "data privacy protection"
results = hybrid_search(
    query=query,
    node_vectors=node_vectors,
    node_texts=node_texts,
    G=G,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    model=model,
    k=3,
    alpha=0.7,  # 70% semantic, 30% keywords
    node_type="control",
)
print(f"Query: '{query}'")
print_results(results)

# Try different queries
queries = [
    "AI model governance",
    "risk management controls",
    "security monitoring requirements",
    "data subject rights",
]

for query in queries:
    print(f"\n{'=' * 80}")
    print(f"QUERY: '{query}'")
    print("=" * 80)
    results = hybrid_search(
        query=query,
        node_vectors=node_vectors,
        node_texts=node_texts,
        G=G,
        sparse_vectors=sparse_vectors,
        sparse_vectorizer=sparse_vectorizer,
        model=model,
        k=3,
        alpha=0.7,
        node_type="control",
    )
    print_results(results)

Query: 'data privacy protection'

Found 3 results:

1. Similarity: 0.9794
   Node: control_30
   Type: control
   Text: The organisation shall implement appropriate technical mechanisms and privacy-enhancing technologies in AI systems to protect personal data and ensure privacy by default. This includes implementing en...
   Domain: Privacy
   Topic: Privacy-Enhancing Technologies and Mechanisms

2. Similarity: 0.8324
   Node: control_27
   Type: control
   Text: The organisation shall implement privacy by design principles in all AI systems, ensuring privacy considerations are embedded from initial planning through system retirement. This includes establishin...
   Domain: Privacy
   Topic: Privacy by Design and Governance

3. Similarity: 0.7496
   Node: control_29
   Type: control
   Text: The organisation shall establish processes to monitor compliance with privacy requirements, detect and respond to privacy incidents, and ensure continuous improvement of privacy controls. This incl

## Compare Dense vs Sparse vs Hybrid

In [36]:
%%time

# Limit results to specific node type (all node types by default)
node_type = None  # "control"
k = 5
alpha = 0.6

# Example 1: Query with specific standard ID (sparse should win)
compare_search_methods(
    query="ISO27001 controls",
    node_vectors=node_vectors,
    node_texts=node_texts,
    G=G,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    model=model,
    k=k,
    alpha=alpha,
)

# Example 2: Conceptual query (dense should win)
compare_search_methods(
    query="protecting sensitive information",
    node_vectors=node_vectors,
    node_texts=node_texts,
    G=G,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    model=model,
    k=k,
    alpha=alpha,
)

# Example 3: Mixed query (hybrid should win)
compare_search_methods(
    query="NIST risk management frameworks",
    node_vectors=node_vectors,
    node_texts=node_texts,
    G=G,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    model=model,
    k=k,
    alpha=alpha,
)


QUERY: 'ISO27001 controls'

1. DENSE ONLY (Semantic):
--------------------------------------------------------------------------------
1. 0.408 | The organisation shall implement and maintain comprehensive physical security controls to protect in...
2. 0.391 | The organisation shall implement mechanisms for meaningful human oversight of AI systems, ensuring h...
3. 0.390 | The organisation shall implement appropriate technical and organisational measures to address identi...

2. SPARSE ONLY (Keywords):
--------------------------------------------------------------------------------
1. 0.211 | The organisation shall implement and maintain comprehensive physical security controls to protect in...
2. 0.195 | The organisation shall conduct regular testing of AI system safety and security controls, including ...
3. 0.150 | The organisation shall implement and maintain comprehensive network security controls to protect aga...

3. HYBRID (alpha=0.6):
-----------------------------------------

# Get subgraph around query results and visualise

### Graph-Based Context Retrieval

Get full context around relevant controls:

```python
subgraph = get_subgraph_around_query("risk management", k=3, hops=1)
```

**Returns:**
- Relevant controls
- Related topics and domains
- All mapped standard references
- Network connections

In [37]:
method_to_test = "hybrid"
possible_method_to_test = ["dense", "sparse", "hybrid"]
if method_to_test not in possible_method_to_test:
    raise ValueError(f"method_to_test has to be one of {possible_method_to_test}")

if method_to_test == "dense":
    # Dense search
    subgraph, results = get_subgraph_around_query(
        query="data privacy",
        G=G,
        search_func=dense_search,
        search_params={
            "node_vectors": node_vectors,
            "node_texts": node_texts,
            "model": model,
            "node_type": "control",
        },
        k=3,
        hops=1,
    )

elif method_to_test == "sparse":
    # Sparse search
    subgraph, results = get_subgraph_around_query(
        query="ISO27001 controls",
        G=G,
        search_func=sparse_search,
        search_params={
            "sparse_vectors": sparse_vectors,
            "sparse_vectorizer": sparse_vectorizer,
            "node_texts": node_texts,
            "node_type": "control",
        },
        k=3,
        hops=1,
    )

elif method_to_test == "hybrid":
    # Hybrid search
    subgraph, results = get_subgraph_around_query(
        query="risk management",
        G=G,
        search_func=hybrid_search,
        search_params={
            "node_vectors": node_vectors,
            "node_texts": node_texts,
            "sparse_vectors": sparse_vectors,
            "sparse_vectorizer": sparse_vectorizer,
            "model": model,
            "alpha": 0.7,
            "node_type": "control",
        },
        k=3,
        hops=1,
    )

In [38]:
print(f"Query: '{query}'")
print(
    f"Subgraph: {subgraph.number_of_nodes()} nodes, {subgraph.number_of_edges()} edges"
)

# Show what's in the subgraph
types_in_subgraph = {}
for node, data in subgraph.nodes(data=True):
    ntype = data.get("type", "unknown")
    types_in_subgraph[ntype] = types_in_subgraph.get(ntype, 0) + 1

print("\nSubgraph composition:")
for ntype, count in types_in_subgraph.items():
    print(f"  {ntype}: {count}")

Query: 'data subject rights'
Subgraph: 24 nodes, 21 edges

Subgraph composition:
  standard: 18
  topic: 3
  control: 3


# Visualization Capabilities

## Interactive Visualization (PyVis)

- Hover to see full control text
- Click to explore connections
- Physics-based layout
- Filterable and zoomable

**Features:**
- Color legend for node types
- Relevance-based sizing
- Relationship labels on edges
- Responsive browser-based interface

In [39]:
# With hybrid search
extract_and_visualize_subgraph(
    query="data privacy protection",
    G=G,
    search_func=hybrid_search,
    search_params={
        "node_vectors": node_vectors,
        "node_texts": node_texts,
        "sparse_vectors": sparse_vectors,
        "sparse_vectorizer": sparse_vectorizer,
        "model": model,
        "alpha": 0.7,
        #'node_type': 'control'
    },
    k=4,
    hops=1,
    output_file="graph_hybrid.html",
)

✓ Interactive graph saved to: graph_hybrid.html
  Nodes: 13
  Edges: 12


# Workflow

In [40]:
semantic_results, expanded, subgraph = search_and_expand_hybrid_filtered(
    query="data privacy protection",
    G=G,
    node_vectors=node_vectors,
    node_texts=node_texts,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    structural_vectors=structural_vectors,
    model=model,
    k_search=5,
    max_hops=10,
    min_structural_sim=0.5,  # Must be structurally similar
    min_semantic_sim=0.5,  # AND semantically relevant to query
    structural_weight=0.5,  # 50-50 balance
    alpha=0.7,  # Weight alpha to attribute to semantic (dense) search, (1 - alpha) for keyword (sparse) search
)

report = generate_report_hybrid_workflow_results(semantic_results, expanded)
print(report)

Stage 1: Hybrid search for 'data privacy protection'...
--------------------------------------------------------------------------------

Found 5 semantically relevant seed nodes:
  1. control_30 (score: 0.981)
     The organisation shall implement appropriate technical mechanisms and privacy-enhancing technologies...
  2. control_27 (score: 0.870)
     The organisation shall implement privacy by design principles in all AI systems, ensuring privacy co...
  3. control_29 (score: 0.799)
     The organisation shall establish processes to monitor compliance with privacy requirements, detect a...
  4. control_19 (score: 0.526)
     The organisation shall protect data throughout its lifecycle using appropriate technical and procedu...
  5. control_28 (score: 0.498)
     The organisation shall implement operational processes for the responsible collection, use, storage,...

Stage 2: Hybrid filtering (structural + semantic)...
  - Max hops: 10
  - Min structural similarity: 0.5
  - Min semant

In [41]:
# Access subgraph data
print(f"\nSubgraph ({subgraph}) node attributes:")
for node in list(subgraph.nodes()):
    print(f"\n* {node}:")
    for key, value in subgraph.nodes[node].items():
        if key not in ["statement"]:  # Skip long text
            print(f"  {key}: {value}")

# Export subgraph if needed
# nx.write_gml(subgraph, "filtered_subgraph.gml")

# Visualise subgraph
visualize_graph_with_pyvis(subgraph, output_file="hybrid_filtered_graph.html")


Subgraph (DiGraph with 13 nodes and 11 edges) node attributes:

* std_EU_AI_ACT_10.5 26.9 27.4:
  type: standard
  standard: EU_AI_ACT
  reference: 10.5 26.9 27.4
  is_seed: False
  seed_node: control_27
  hop_distance: 1
  direction: successor
  structural_similarity: 0.9573681354522705
  semantic_similarity: 0.5740383267402649
  combined_score: 0.7657032012939453

* topic_Privacy-Enhancing Technologies and Mechanisms:
  type: topic
  name: Privacy-Enhancing Technologies and Mechanisms
  is_seed: False
  seed_node: control_30
  hop_distance: 1
  direction: predecessor
  structural_similarity: 0.9850808382034302
  semantic_similarity: 0.6742457151412964
  combined_score: 0.8296632766723633

* topic_Privacy Compliance and Monitoring:
  type: topic
  name: Privacy Compliance and Monitoring
  is_seed: False
  seed_node: control_29
  hop_distance: 1
  direction: predecessor
  structural_similarity: 0.9784229397773743
  semantic_similarity: 0.645642876625061
  combined_score: 0.81203293800

# Results exploration

## Graph statistics

In [42]:
# Usage
stats = get_subgraph_stats(
    subgraph, include_node_breakdown=True, include_centrality=True, include_paths=True
)
print_subgraph_stats(stats, verbose=True)


SUBGRAPH STATISTICS

📊 Basic Metrics:
   nodes: 13
   edges: 11
   density: 0.07051282051282051
   is_connected: False

🔗 Degree Statistics:
   average: 1.69
   max: 4.00
   min: 1.00
   median: 1.00

🏷️  Node Types:
   standard: 2
   topic: 5
   control: 5
   domain: 1

🎯 Node Roles:
   seed: 5
   found: 8
   intermediate: 0

⭐ Most Central Nodes:
   By Degree:
      domain_Privacy: 0.333
      control_27: 0.250
      topic_Privacy-Enhancing Technologies and Mechanisms: 0.167
      topic_Privacy Compliance and Monitoring: 0.167
      topic_Personal Data Management: 0.167
   By Betweenness:
      control_27: 0.030
      topic_Privacy by Design and Governance: 0.023
      topic_Privacy-Enhancing Technologies and Mechanisms: 0.008
      topic_Privacy Compliance and Monitoring: 0.008
      topic_Personal Data Management: 0.008

🔗 Edge Types:
   has_control: 5
   contains: 4
   maps_to: 2

📈 Relevance Scores:
   avg_score: 0.809
   max_score: 0.839
   min_score: 0.766
   nodes_with_scores

## Node importance ranking

In [43]:
# Usage examples
rankings = rank_nodes_by_importance(
    subgraph,
    methods="all",  # or ['pagerank', 'degree', 'relevance']
    top_k=10,
    aggregate="average",  # or 'max' or {'pagerank': 0.4, 'degree': 0.3, 'relevance': 0.3}
)

print_node_rankings(rankings, subgraph, show_details=True)


NODE IMPORTANCE RANKINGS

Ranked 13 nodes using: pagerank, degree, betweenness, eigenvector, relevance


📊 PAGERANK (Top 5):
   1. control_28 (control): 0.0988
   2. control_30 (control): 0.0988
   3. control_29 (control): 0.0988
   4. control_27 (control): 0.0988
   5. std_EU_AI_ACT_10.5 26.9 27.4 (standard): 0.0907

📊 DEGREE (Top 5):
   1. domain_Privacy (domain): 0.3333
   2. control_27 (control): 0.2500
   3. topic_Privacy-Enhancing Technologies and Mechanisms (topic): 0.1667
   4. topic_Privacy Compliance and Monitoring (topic): 0.1667
   5. topic_Personal Data Management (topic): 0.1667

📊 BETWEENNESS (Top 5):
   1. control_27 (control): 0.0303
   2. topic_Privacy by Design and Governance (topic): 0.0227
   3. topic_Privacy-Enhancing Technologies and Mechanisms (topic): 0.0076
   4. topic_Privacy Compliance and Monitoring (topic): 0.0076
   5. topic_Personal Data Management (topic): 0.0076

❌ CLOSENESS: graph is not connected enough to compute closeness centrality

📊 EIGENVECTOR

In [46]:
# Compare specific node
# Check if node exists first
node_to_check = "control_28"
if node_to_check in subgraph:
    node_comparison = compare_node_importance(node_to_check, rankings)
    if node_comparison:
        print(f"\nHow {node_to_check} ranks:")
        for method, info in node_comparison.items():
            print(f"  {method}: #{info['rank']} (score: {info['score']:.3f})")
    else:
        print(f"\{node_to_check} not in top rankings")
else:
    print(f"\{node_to_check} not in subgraph")


How control_28 ranks:
  pagerank: #1 (score: 0.099)
  degree: #9 (score: 0.083)
  betweenness: #8 (score: 0.000)
  eigenvector: #3 (score: 0.003)


In [47]:
filtered_rankings, full_rankings = rank_nodes_by_importance_with_context(
    subgraph,
    focus_type="control",
    methods="all",
    top_k=10,
    aggregate="average",  # or 'max' or {'pagerank': 0.4, 'degree': 0.3, 'relevance': 0.3}
)

print("\n=== CONTROL NODE RANKINGS (computed on full graph) ===")
print_node_rankings(filtered_rankings, subgraph)


=== CONTROL NODE RANKINGS (computed on full graph) ===

NODE IMPORTANCE RANKINGS

Ranked 13 nodes using: pagerank, degree, betweenness, eigenvector, relevance


📊 PAGERANK (Top 5):
   1. control_28 (control): 0.0988
   2. control_30 (control): 0.0988
   3. control_29 (control): 0.0988
   4. control_27 (control): 0.0988
   5. control_19 (control): 0.0900

📊 DEGREE (Top 5):
   1. control_27 (control): 0.2500
   2. control_19 (control): 0.0833
   3. control_28 (control): 0.0833

📊 BETWEENNESS (Top 5):
   1. control_27 (control): 0.0303
   2. control_19 (control): 0.0000
   3. control_28 (control): 0.0000
   4. control_30 (control): 0.0000

❌ CLOSENESS: graph is not connected enough to compute closeness centrality

📊 EIGENVECTOR (Top 5):
   1. control_28 (control): 0.0026
   2. control_30 (control): 0.0026
   3. control_29 (control): 0.0026
   4. control_27 (control): 0.0026
   5. control_19 (control): 0.0000

📊 RELEVANCE (Top 5):

⭐ COMBINED RANKING (Top 10):
   1. control_27 (control): 

In [48]:
# Debug why rankings might be zero for some nodes in some metrics
diagnose_rankings(subgraph, "control")

Total nodes: 13
Filtered nodes (control): 5
Filtered subgraph connected: False
Filtered subgraph edges: 0
Isolated nodes: 5/5

⚠️  WARNING: All filtered nodes are isolated!
   Recommendation: Don't filter by node_type for ranking.
   Rank on full graph, then filter results for display.


## Explain node retrieval

In [51]:
# For a specific node
explanation = explain_retrieval(
    node_id="topic_Data Security",  # 'control_28',
    query="data privacy protection",
    subgraph=subgraph,
    node_vectors=node_vectors,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    structural_vectors=structural_vectors,
    model=model,
)
print_explanation(explanation, verbose=True)


RETRIEVAL EXPLANATION: topic_Data Security

Node Type: topic
Query: 'data privacy protection'

🎯 Reason: FOUND VIA GRAPH TRAVERSAL
   From seed: control_19
   Distance: 1 hops (predecessor)
   Structural similarity: 0.982
   Semantic similarity: 0.599
   Combined score: 0.790

📝 Semantic Similarity: 0.599
   Moderate similarity

🔍 Keyword Matching: 0.219
   Very low similarity
   Matching terms: data
   Top TF-IDF terms in node:
      - topic data: 0.665
      - security: 0.483
      - data: 0.453

🕸️  Structural Similarity:
   Shares graph structure with seed nodes
   Similar to seed nodes:
      - control_19: 0.982
      - control_27: 0.786
      - control_28: 0.748

📄 Content Preview:
   Data Security



In [52]:
explanations = explain_top_results(
    query="data privacy protection",
    results=semantic_results,
    subgraph=subgraph,
    node_vectors=node_vectors,
    sparse_vectors=sparse_vectors,
    sparse_vectorizer=sparse_vectorizer,
    structural_vectors=structural_vectors,
    model=model,
    top_k=3,
)


RETRIEVAL EXPLANATION: control_30

Node Type: control
Query: 'data privacy protection'

🎯 Reason: SEED NODE
   Initial search score: 0.981
   - Semantic component: 1.000
   - Keyword component: 0.938

📝 Semantic Similarity: 0.661
   Moderate similarity

🔍 Keyword Matching: 0.438
   Low similarity
   Matching terms: privacy, data
   Top TF-IDF terms in node:
      - privacy: 0.371
      - data: 0.321
      - mechanisms: 0.267

🕸️  Structural Similarity:
   Shares graph structure with seed nodes
   Similar to seed nodes:
      - control_27: 0.821
      - control_28: 0.815
      - control_29: 0.775

📄 Content Preview:
   The organisation shall implement appropriate technical mechanisms and privacy-enhancing technologies in AI systems to protect personal data and ensure privacy by default. This includes implementing en...


RETRIEVAL EXPLANATION: control_27

Node Type: control
Query: 'data privacy protection'

🎯 Reason: SEED NODE
   Initial search score: 0.870
   - Semantic component: 0.8

# Take exploration results and ask LLM to answer original client query only using subgraph

In [53]:
api_keys = {
    "Anthropic": os.getenv("ANTHROPIC_API_KEY"),
    "Mistral": os.getenv("MISTRAL_API_KEY"),
    "OpenAI": os.getenv("OPENAI_API_KEY"),
}

## Create prompt

In [54]:
# Complete prompt
prompt = create_llm_prompt_with_graph(
    query="data privacy protection",
    subgraph=subgraph,
    report=report,
    format="natural",  # or 'markdown'
)

# Send to LLM
print(prompt)

# Save to file
with open("llm_prompt.txt", "w") as f:
    f.write(prompt)

# Graph-Based Query Response

## User Query
"data privacy protection"

## Search Results
HYBRID-FILTERED WORKFLOW RESULTS

1. SEED NODE (Hybrid Search Match):
   Node: control_30
   Semantic Score: 0.981
   Text: The organisation shall implement appropriate technical mechanisms and privacy-enhancing technologies in AI systems to protect personal data and ensure...
   Domain: Privacy
   Topic: Privacy-Enhancing Technologies and Mechanisms

   HYBRID-FILTERED NEIGHBORS:
   (Must pass BOTH structural AND semantic thresholds)
   1. domain_Privacy (predecessor, 2 hops)
      Combined: 0.844
      └─ Structural: 0.928
      └─ Semantic: 0.761
      Type: domain
      Domain: Privacy. Topics: Privacy by Design and Governance, Personal Data Managem...
   2. topic_Privacy-Enhancing Technologies and Mechanisms (predecessor, 1 hops)
      Combined: 0.830
      └─ Structural: 0.985
      └─ Semantic: 0.674
      Type: topic
      Topic: Privacy-Enhancing Technologies and Mechanisms. Domain: Privac

## Query LLM

In [55]:
%%time

provider = "Anthropic"

result = query_llm(
    prompt=prompt,
    # system_prompt=system_prompt,
    provider=provider,
    api_key=api_keys[provider],
    temperature=0.2,
)

CPU times: user 195 ms, sys: 18.8 ms, total: 214 ms
Wall time: 9.46 s


In [56]:
display(Markdown(result))

# Response to Query: "data privacy protection"

## Key Insights

Based strictly on the graph data, here's a comprehensive analysis of data privacy protection:

### Core Components
The graph reveals four primary aspects of data privacy protection:

1. Privacy-Enhancing Technologies
- Represented by control_30
- Focuses on implementing technical mechanisms to protect personal data
- Semantic Score: 0.981 (highest relevance)

2. Privacy by Design
- Represented by control_27
- Embeds privacy considerations from initial system planning
- Semantic Score: 0.870

3. Privacy Compliance and Monitoring
- Represented by control_29
- Establishes processes to:
  * Monitor privacy requirements
  * Detect privacy incidents
  * Ensure continuous compliance
- Semantic Score: 0.799

4. Personal Data Management
- Represented by control_28
- Covers responsible:
  * Collection
  * Use
  * Storage
  * Disposal of personal data
- Semantic Score: 0.498

### Interconnected Domain
All these components are interconnected within the "domain_Privacy" node, suggesting a holistic approach to data privacy protection.

### Relevant Standards
The graph links these privacy controls to two standards:
- EU AI Act (sections 10.5, 26.9, 27.4)
- SOC2 (sections P1.1, P1.2, P1.3)

## Key Relationships
- Privacy domain contains 4 key topics
- Each topic has associated control mechanisms
- Controls map to specific regulatory standards

## Recommended Actions
Based on the graph, organizations should:
1. Implement privacy-enhancing technologies
2. Adopt privacy by design principles
3. Establish monitoring and compliance processes
4. Manage personal data responsibly
5. Align with EU AI Act and SOC2 standards

The graph emphasizes a comprehensive, proactive approach to data privacy protection across technical, operational, and governance dimensions.

In [57]:
print("Notebook finished !")

Notebook finished !
